In [1]:
import shutil
import os

import pandas as pd
from sklearn.utils import shuffle
import matplotlib.patches as patches
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2


import keras
import keras.backend as K
from keras.models import Model
from keras import backend as K
from keras.utils import np_utils
from keras.optimizers import SGD, Adam
from keras.datasets import cifar10
from keras.layers.core import Layer
from keras.models import model_from_json
from keras.callbacks import LearningRateScheduler
from keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Input, concatenate, GlobalAveragePooling2D, AveragePooling2D,Flatten

import numpy as np 
import math 

Using TensorFlow backend.


In [ ]:
#save test images in a separate folder (test_file)

def save_test():
    df_2 = pd.read_csv('test.csv')
    testfileslist = df_2['image_name'].tolist()
    testfileslist = ["F:\Flipkart Grid\Round 2\images/" + s for s in testfileslist]
    dest = "test_file/"
    print(testfileslist[2990])
    for file_name in testfileslist:
        shutil.copy(file_name, dest)
            
save_test()

In [5]:
# Load the trained model and predict on test data

initial_lrate = 0.025
sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)
json_file = open('model.json', 'r')
model = json_file.read()
json_file.close()
model = model_from_json(model)
model.load_weights('F:\Flipkart Grid\Round 2/model.h5')
model.compile(loss='mean_absolute_error',  optimizer=sgd)

In [ ]:
#Function to load test data in batches

def load_test_data(index, minibatchsize, testfileslist, last=False):
    images = []
    if last == False:
          batchlist = testfileslist[index*minibatchsize:(index+1)*minibatchsize]
    else:
          batchlist = testfileslist[index*minibatchsize:]
    for i,f in enumerate(batchlist):
        image = Image.open(f)
        image = image.resize((224, 224))
        image = np.array(image)
        images.append(image)
    images = np.array(images)

    X_train = images.astype('float32')
    X_train = (X_train) / 255.0

    return X_train

# Function to get predictions from Test Data 
# Note=> (Please modify the path to the required one while predicting from another machine)
def eval_test():
    minibatchsize = 50
    df_2 = pd.read_csv('test.csv')
    testfileslist = df_2['image_name'].tolist()
    testfileslist = ["F:\Flipkart Grid\Round 2\images/" + s for s in testfileslist]

    for j in range(1):
        for i in range(int(len(testfileslist)/minibatchsize)):
            X_test = load_test_data(i, minibatchsize, testfileslist, last = False)
            predict = model.predict(X_test, batch_size=None, verbose=0, steps=None)            
            if i ==0 : predcoords = predict
            else: predcoords = np.concatenate((predcoords,predict),axis=1)
            if i%10 == 0: print(np.shape(predcoords))
    X_test = load_test_data(i, minibatchsize, testfileslist, last = True)
    predict = model.predict(X_test, batch_size=None, verbose=0, steps=None)
    predcoords = np.concatenate((predcoords,predict),axis=1)
    
    return predcoords

testpredcoords = eval_test()

In [7]:
#Convert Predictions to the required format and save as CSV file for submission

def testcsv():
    x1 = (testpredcoords[0,:,0]-(testpredcoords[1,:,0]/2))*640
    y1 = (testpredcoords[0,:,1]-(testpredcoords[1,:,1]/2))*480
    x2 = (testpredcoords[0,:,0]+(testpredcoords[1,:,0]/2))*640
    y2 = (testpredcoords[0,:,1]+(testpredcoords[1,:,1]/2))*480
    df_2 = pd.read_csv('test.csv')
    testcsv = pd.DataFrame({'image_name':df_2['image_name'], 'x1':x1[:12815], 'y1':y1[:12815], 'x2':x2[:12815], 'y2':y2[:12815]})
    testcsv.to_csv('test_submission.csv', index=False)
    
testcsv()

                               image_name          x1          x2          y1  \
0               1474723840903DSC08089.png  230.964600  457.465820   87.566612   
1        1473231475010DeeplearnS11276.png   69.967461  571.169250  140.875000   
2  JPEG_20161205_135307_1000155917326.png  134.421707  496.897827   52.300659   
3  JPEG_20160711_123440_1000518778437.png  225.641663  453.102692   93.387581   
4   JPEG_20160803_115329_100034020722.png  123.702560  512.057495   33.830811   

           y2  
0  406.314606  
1  345.206177  
2  428.468903  
3  414.372498  
4  429.566223  


(2, 12865, 2)

In [ ]:
# Plot the predictions on Test Data to check the Model

def plot_image(i):
    df_2 = pd.read_csv('test.csv')
    testfileslist = df_2['image_name'].tolist()
    testfileslist = ["F:\Flipkart Grid\Round 2\images/" + s for s in testfileslist]
    img = cv2.imread(testfileslist[i])
    fig,ax = plt.subplots(1)
    ax.imshow(img)
    x = (testpredcoords[0][i][0]-(testpredcoords[1][i][0]/2))*640
    y = (testpredcoords[0][i][1]-(testpredcoords[1][i][1]/2))*480
    w = (testpredcoords[1][i][0])*640
    h = (testpredcoords[1][i][1])*480
    rect = patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='g',facecolor='none')
    ax.add_patch(rect)
    plt.show()
    
plot_image(10)
plot_image(20)
plot_image(30)
plot_image(40)